In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [624 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,473 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,278 kB]
Get:13 http://archive.ubunt

Exception: ignored

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
hs_df.createOrReplaceTempView('home_sales')
hs_df.show()


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query_1  = """

SELECT
  YEAR(date) AS Sale_Year,
  ROUND(AVG(price), 2) AS Avg_Price

FROM home_sales

WHERE bedrooms = 4

GROUP BY Sale_Year

ORDER BY Avg_Price DESC
"""

spark.sql(query_1).show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query_2  = """

SELECT
  YEAR(date_built) AS Year_Built,
  ROUND(AVG(price), 2) AS Avg_Price

FROM home_sales

WHERE bedrooms = 3

AND bathrooms = 3

GROUP BY year(date_built)

ORDER BY Avg_Price DESC
"""

spark.sql(query_2).show()



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query_3  = """

SELECT
  YEAR(date_built) AS Year_Built,
  ROUND(AVG(price), 2) AS Avg_Price

FROM home_sales

WHERE bedrooms = 3

AND bathrooms = 3

AND floors = 2

AND sqft_living >= 2000

GROUP BY year(date_built)

ORDER BY Avg_Price DESC
"""

spark.sql(query_3).show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()


query_4  = """

SELECT
  VIEW,
  ROUND(AVG(price), 2) AS Avg_Price

FROM home_sales

GROUP BY VIEW

ORDER BY VIEW DESC
"""

spark.sql(query_4).show()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.


spark.table("home_sales").cache()

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()


spark.sql("SELECT view, ROUND(AVG(price), 2) FROM home_sales WHERE  price>=350000 GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

house_data.write.parquet('parquet_home_sales', mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.

p_data=spark.read.parquet('parquet_home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.

p_data.createOrReplaceTempView('p_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price), 2) FROM p_sales WHERE  price>=350000 GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')